Links
- [DSPy Documentation](https://dspy.ai/#__tabbed_1_4)


Double Saved to `obsidian-llm-tool-use` Github repo and my personal note vault in obsidian via:
```bash
jupytext --to markdown dspy_modules/intro_to_dspy.ipynb -o ~/Obsidian/Notes\ Vault/intro_to_dspy.md
```

First, import the package and setup your llm calling configuration. For this, we'll be using ollama.

Make sure to spin up your ollama server using
```bash
ollama start
```

In [3]:
import dspy
lm = dspy.LM('openai/qwen2.5:7b-instruct-q4_K_M', api_base='http://localhost:11434/v1', api_key='')
dspy.configure(lm=lm)

Lets do the basic prompt-response: just use the `lm` as a function!

In [4]:
lm("Say this is a test!", temperature=0.7)  # => ['This is a test!']

["Sure, this certainly seems like a test! How can I help you with it? Whether it's a language comprehension test, a creativity challenge, or something else entirely, feel free to provide more details so I can assist you better."]

Super compact syntax.

You could just use this as a nice way to make your LLM calls a bit more pythonic.

You can also send using the chat completions formatting.

In [5]:
lm(messages=[{"role": "user", "content": "Say this is a test!"}])  # => ['This is a test!']

["Of course! This is definitely a test. How can I help you with it? Whether you're looking to practice language skills, need some questions answered, or just want to see how things work, feel free to let me know what you'd like to do!"]

Ok now to the first main topic:

## Modules

Modules help you describe AI behavior as *code*. not *strings*.

You specify a *Signature*: a string that defines an action via inputs and outputs behavior: `"question -> answer: float"`

Then you select a *Module* to assign a strategy for invoking the LLM. `Predict` is the simplest one.

In [32]:
solve_math = dspy.Predict("question -> answer: float")
result = solve_math(question="What is 1 + 1?")
print(result.completions)



Completions(
    answer=[2.0]
)


A Module:
- wraps a signature.
- is callable
- carries "learnable parameters" that DSPy can run optimization on.
- composes: modules call other modules, can be stored as `json`, or be nested inside larger `dspy.Program` graphs.
- persists: `module.save()`/`load()` for controlling state.


There's a few really powerful primative ones already implemented, like `dspy.ChainOfThought`. It automatically:
1. Inserts an instruction telling the LLM to show its reasoning.
2. Adds an implicit extra output field called `reasoning`.
3. Returns both the reasoning and the final answer, while still respecting the original signature.

So that's why it won't be very good for:

In [16]:
result = solve_math(question="What is the third root of 963261?")
print(result.completions)

Completions(
    reasoning=['To find the third root of 963261, we need to calculate 963261^(1/3).'],
    answer=[45.0]
)


But you can get pretty creative with the signatures.  LLM act as this universal function approximator written via English.  Modules try to shape that approximator into a math function.

In [ ]:
solve_riddle = dspy.Predict("riddle -> answer")
print(solve_riddle(riddle="What has keys but can't open locks?").answer)

keyboard


In your *Signature* you can list multiple fields: `"context: list[str], question -> answer"` or omit the types if they're strings.

In [ ]:
emojify = dspy.Predict("story -> emoji_sequence")

story="""You're walking in the woods
There's no one around and your phone is dead
Out of the corner of your eye you spot him
Shia LaBeouf

He's following you, about 30 feet back
He gets down on all fours and breaks into a sprint
He's gaining on you
Shia LaBeouf

You're looking for you car but you're all turned around
He's almost upon you now
And you can see there's blood on his face
My God, there's blood everywhere!

Running for you life (from Shia LaBeouf)
He's brandishing a knife (it's Shia LaBeouf)
Lurking in the shadows
Hollywood superstar Shia LaBeouf

Living in the woods (Shia LaBeouf)
Killing for sport (Shia LaBeouf)
Eating all the bodies
Actual cannibal Shia LaBeouf"""

emoji_sequence = emojify(story=story)
print(emoji_sequence.emoji_sequence)

Completions(
    emoji_sequence=['🌳📱🏃\u200d♂️👀🧱🔥🔪 manhuntsurvivorHunterShiaLaBeoufwoods CannibalShiaLaBeouf']
)
🌳📱🏃‍♂️👀🧱🔥🔪 manhuntsurvivorHunterShiaLaBeoufwoods CannibalShiaLaBeouf


In [ ]:
translate = dspy.Predict("string -> italian")
translation = translate(string=story)
print(translation.italian)

Stai camminando nel bosco
Non c'è nessuno intorno e il tuo cellulare non funziona
Dalla parte del tuo occhio destro lo vedi
Shia LaBeouf

Lo sta seguendo, a circa 30 piedi di distanza
Si mette in quattro zampe e scatta in corsa
Sta guadagnando terreno
Shia LaBeouf

Cercavi la tua macchina ma ti sei girato tutto intorno
Ora è quasi sopra di te
E puoi vedere che c'è sangue sul suo viso
Dio mio, c'è sangue dappertutto!

Correndo per la tua vita (da Shia LaBeouf)
Sta brandendo un coltello (è Shia LaBeouf)
Si aggira nelle ombre
Superstite hollywoodiano Shia LaBeouf

Vivere nel bosco (Shia LaBeouf)
Uccidendo a sangue caldo (Shia LaBeouf)
Mangiando tutti i corpi
Cannibale reale Shia LaBeouf


### Single-shot predictors

There's `Predict`, `ChainOfThought`, and `ChainOfThoughtWithHint` as well.

In [ ]:
find_time_complexity = dspy.ChainOfThought("function -> time_complexity")

def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)
    
complexity = find_time_complexity(function=fibonacci)
print(complexity.completions)

Completions(
    reasoning=["The Fibonacci function is typically implemented using recursion or iteration. In this case, since no specific implementation details are provided, we will consider a basic recursive approach which has an exponential time complexity due to the repeated calculations of the same subproblems.\n\nFor the iterative approach, it would have a linear time complexity. However, without knowing the exact implementation, we'll assume the worst-case scenario for simplicity."],
    time_complexity=['O(2^n)']
)


In [ ]:
cot_hint = dspy.ChainOfThoughtWithHint("question -> answer: float")
prediction = cot_hint(question="What is 16 x 17?", hint="16x10=160 and 16x7=112")  
print(prediction.reasoning)
print("----")
print(prediction.answer)

We can use the distributive property of multiplication over addition to solve this problem. The hint suggests breaking down 17 into 10 + 7, so we have:

\[ 16 \times 17 = 16 \times (10 + 7) = (16 \times 10) + (16 \times 7) \]

Given that \(16 \times 10 = 160\) and \(16 \times 7 = 112\), we can add these two results together:

\[ 160 + 112 = 272 \]

Therefore, the answer is 272.
----
272.0


### Multi-shot Predictors

Some built-in Modules use multiple LLM calls and tools to iteratively improve responses.

`ReAct`: implements a *ReAct* agent pattern: the LLM alternates between thinking and calling user-supplied tooks, and stops when it fills the Signature. Used for search-and-answer agents, code-execution helpers, custom tool use.

In [40]:
# ReAct tool counts number of letter occurances in a string
def count_letters(string: str) -> dict:
    counts = {}
    for letter in string:
        if letter.isalpha():
            counts[letter] = counts.get(letter, 0) + 1
    return counts

question_answerer = dspy.ReAct("question -> answer",tools=[count_letters],max_iters=3)

print(question_answerer(question="How many R's in the word strawberry?"))


Prediction(
    trajectory={'thought_0': 'To find out how many R\'s are in the word "strawberry", I can use the count_letters tool.', 'tool_name_0': 'count_letters', 'tool_args_0': {'string': 'strawberry'}, 'observation_0': {'s': 1, 't': 1, 'r': 3, 'a': 1, 'w': 1, 'b': 1, 'e': 1, 'y': 1}, 'thought_1': 'The count_letters tool returned that the word "strawberry" contains 3 \'r\'s. Since I have all the information needed to answer the question, I can now finish.', 'tool_name_1': 'finish', 'tool_args_1': {}, 'observation_1': 'Completed.'},
    reasoning='To determine how many R\'s are in the word "strawberry", I used a tool to count each letter. The result showed that there are 3 \'r\'s.',
    answer="There are 3 R's in the word strawberry."
)


`ProgramOfThought`: ask LLM to write a python program, executes it, then passes result back into the answer. 

It relies on `Deno`, which needs to be installed using:

```bash
brew install deno
```

In [7]:
pot_math = dspy.ProgramOfThought("question -> answer", max_iters=3)
print(pot_math(question="What is the third root of 963261?").answer)



2025/05/12 20:22:39 ERROR dspy.predict.program_of_thought: Error in code execution: Invalid Python syntax. message: 


KeyboardInterrupt: 

`MultiChainComparison`: Spins up `M` separate `ChainOfThought` traces, asks the LLM to vote-critique, and returns the best. Fastest way to logarithmically scale intelligence.

### Your Own Modules and Signatures

Starting from the built-in ones, you can construct your own modules that include multiple LLM calls and complicated flows.

Custom signatures work similarly. You can define multiple `InputField()` and `OutputField()`'s.



In [33]:
#first, we'll tell it our birthday
#and it figures out our sign
class ZodiacSignature(dspy.Signature):
    birth_day = dspy.InputField()
    sign= dspy.OutputField(desc="Aries, Taurus, …, Pisces")


#then it creates a horoscope.
class HoroscopeSignature(dspy.Signature):
    sign= dspy.InputField(desc='Zodiac sign')
    current_date= dspy.InputField(desc='In YYYY-MM-DD format')
    horoscope= dspy.OutputField(desc="One-paragraph horoscope")
    lucky_numbers= dspy.OutputField(desc="Comma-separated lucky numbers")


In [ ]:
from datetime import date


class HoroscopeFromBirthday(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict_sign = dspy.Predict(ZodiacSignature)
        self.create_horoscope=dspy.ChainOfThought(HoroscopeSignature)

    def forward(self, birth_day:str):
        #get the zodiac sign
        sign_pred = self.predict_sign(birth_day=birth_day)
        sign = sign_pred.sign
        #create the horoscope
        today = date.today()
        horo_pred = self.create_horoscope(sign=sign,current_date=today)



        #Prediction acts as the return type. A dictionary wrapper
        return dspy.Prediction(
            sign = sign,
            horoscope_reasoning = horo_pred.reasoning,
            horoscope = horo_pred.horoscope,
            lucky_numbers = horo_pred.lucky_numbers
        )


zodiac_finder=HoroscopeFromBirthday()
print(zodiac_finder(birth_day="I was born on december 25, 2000"))
        

Prediction(
    sign='Capricorn',
    horoscope_reasoning='As a Capricorn, you are known for your discipline and ambition. On May 12, 2025, Mercury is in Gemini, which can bring communication and social activities to the forefront of your life. This transit might affect how you interact with colleagues or friends.',
    horoscope='With Mercury in Gemini, expect a busy day filled with conversations and exchanges of ideas. Your focus on communication could lead to new connections or insights that align with your career goals. Stay open to networking opportunities as they can be beneficial for your professional growth. However, be mindful of multitasking; while it may seem like you have many balls in the air, try to keep them organized to avoid any mix-ups.',
    lucky_numbers='8, 23, 37, 41, 59'
)


Here's a more complicated one:

In [22]:
class FindInspo(dspy.Signature):
    topic   = dspy.InputField()
    blurbs  = dspy.OutputField(format=list, desc="3-5 catchy micro-blurbs")

class DraftPlot(dspy.Signature):
    topic   = dspy.InputField()
    blurbs  = dspy.InputField(format=list)
    outline = dspy.OutputField(desc="bullet outline of the plot")

class InsertPuzzle(dspy.Signature):
    outline = dspy.InputField()
    story   = dspy.OutputField(desc="short story that embeds a logic puzzle")

class BuildQuiz(dspy.Signature):
    story      = dspy.InputField()
    questions  = dspy.OutputField(
        format=list,
        desc="list of (Q, choices:list, correct:str) tuples")

class TeaseTweet(dspy.Signature):
    story = dspy.InputField()
    tweet = dspy.OutputField(desc="≤280-char teaser")


# ── 2. Custom module (3 distinct DSPy modules) ───────────────────
class StoryPuzzleLab(dspy.Module):
    """
    Modules used →
      ① ChainOfThought         – inspirations & quiz
      ② ChainOfThoughtWithHint – plot expansion
      ③ ProgramOfThought       – code-driven puzzle generation
    """

    def __init__(self):
        super().__init__()
        self.inspirer = dspy.ChainOfThought(FindInspo, n=1)
        self.plotter  = dspy.ChainOfThoughtWithHint(DraftPlot)
        self.puzzler  = dspy.ProgramOfThought(InsertPuzzle, max_iters=3)
        self.quizzer  = dspy.ChainOfThought(BuildQuiz)
        self.teaser   = dspy.ChainOfThought(TeaseTweet, max_len=70)  # ≈ tweet

    def forward(self, topic: str):
        blurbs   = self.inspirer(topic=topic).blurbs
        outline  = self.plotter(topic=topic, blurbs=blurbs).outline
        story    = self.puzzler(outline=outline).story
        quiz     = self.quizzer(story=story).questions
        tweet    = self.teaser(story=story).tweet

        return dict(story=story, quiz=quiz, tweet=tweet, inspo=blurbs)



lab = StoryPuzzleLab()
res = lab("Lost temple, steam-age explorers")
print("\n— INSPIRATION —\n", res["inspo"])
print("\n— STORY —\n",res["story"])
print("\n— QUIZ —\n",res['quiz'])
print("\n— TWEET —\n", res["tweet"])

2025/05/12 20:51:42 ERROR dspy.predict.program_of_thought: Error in code execution: Invalid Python syntax. message: 
2025/05/12 20:52:22 ERROR dspy.predict.program_of_thought: Error in code execution: Invalid Python syntax. message: 
2025/05/12 20:53:19 ERROR dspy.predict.program_of_thought: Error in code execution: Invalid Python syntax. message: 


RuntimeError: Max hops reached. Failed to run ProgramOfThought: Invalid Python syntax. message: 

In a steam-age world, ancient civilizations are being rediscovered. Dr. Amelia Hartley, equipped with a steam-powered microscope, Eliot Blackwood with his clockwork compass, and Lila Voss with her invisibility cloak form an exploratory team. They stumble upon clues leading to a lost temple, filled with historical artifacts and puzzles. Navigating through the temple's traps using Dr. Hartley's microscope to identify safe paths, solving riddles that unlock secret passages with Eliot's compass, and avoiding detection by temple guardians with Lila's cloak are key parts of their journey. A confrontation with a mechanical guardian within the temple tests their skills and gadgets. Uncovering the secrets of the lost civilization, they decide to share knowledge responsibly.


In [18]:
type(res['quiz'])

str

## So What?: Optimizers

The syntax is nice and simple and whatever but what's so special about DSPy?

Optimizers are applied at every system prompt that's contained withing your graph of agentic LLM calls. It mutates all the *learnable parameters*: prompt templates, demonstration pools, adaptor weights. It runs a *generate* > *score* > *selectt* loop. It proposes new demos or instructions, runs the module, evaluates the output against a metric function, and keeps the best variants for the next time around. 

Each Optimizer accepts three arguments: 
- a `Module`
- a metric function that returns a float
- train/validation data: 10-300 `Examples`

Optimizers can be saved to json using `save()` and autologged to MLFlow.


## Going Larger: Programs